<a href="https://colab.research.google.com/github/david-j-cox/Man-vs-Machine/blob/master/add_re288_col.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
# Set working directory
from google.colab import drive
drive.mount('/content/gdrive')
%cd './gdrive/My Drive/GME & MLB/Scripts and Notebooks/'

Mounted at /content/gdrive
/content/gdrive/My Drive/GME & MLB/Scripts and Notebooks


In [ ]:
# Connect GPU, if desired
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [2]:
# Connect TPU, if desired
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.7.0
Running on TPU  ['10.102.60.210:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.102.60.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.102.60.210:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
# Packages and modules we'll use
# System
import glob
from google.colab import files
import zipfile, io, os
import warnings
import time

# Data manipulation
import pandas as pd
import numpy as np
from RE288 import RE288_calc

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display, clear_output

print("Packages loaded")

Packages loaded


In [4]:
# Read in RE288 scores
re288_dict = pd.read_csv('../Data/01_raw/RE288_NL.csv').drop(['Unnamed: 0'], axis=1)

### Functions

In [5]:
# Function to add re288 vals to dataframes
def add_gs288(df, year):
  """
  Add a column with the game state for calculating re288 value corresponding to each pitch context in the dataset.
  -----
  Params:
    df: dataframe
      - Dataframe containing the data needed to find the re288 value to map. 
    year: int
      - The year within the dataframe you want to isolate for batch computing. 
  -----
  Returns: 
    The original dataframe with an added column containing the game state and the 
    mapped re288 value for each pitch. 
  """
  # Create RE288 dict
  re288_vals = re288_dict[re288_dict['Year']==year]
  vals_288 = dict(zip(re288_dict['Game_State'], re288_dict['NL_RE288']))

  # Empty lists to store game state and RE288 vals
  gs_vals = []
  re288 = []

  for i in range(len(df)):    # Iterate through dataframe
    # Get pitch context values as keys to get re288 value
    balls = df['balls'][i]
    strikes = df['strikes'][i]
    outs = df['outs_when_up'][i]  
    first = df['on_1b'][i]
    second = df['on_2b'][i]
    third = df['on_3b'][i]
    
    # Pass pitch context keys to re288_calc function
    val = RE288_calc(balls=balls, strikes=strikes, outs=outs, runner_first=first, 
                    runner_second=second, runner_third=third)
    re288_val = vals_288.get(val)
    
    # Add to empty lists for short-term storage
    gs_vals.append(val)
    re288.append(re288_val)

    # Providing notebook user with updating progress in loop. 
    if i%100==0:
      clear_output()
      print(f'Season: {year}\n{i} of {len(df)} completed ({int((i/len(df))*100)}%)')

  # Add data to original passed dataframe    
  df['gs_288'] = gs_vals
  df['re288'] = re288
  return df

# Function to get change in re288 pre-post pitch
def re288_change(df, year):
  """
  Add two columns with the change in re288 value resulting each pitch in the dataset.
    1.) Change in re288 on a Continuous scale. 
    2.) Change in re288 on an ordinal scale (-1=worse; 0=no change; 1=better)
  -----
  Params:
    df: dataframe
      - Dataframe containing the data with the re288 states and cols necessary 
        to temporally sort the data. 
    year: int
      - The year within the dataframe you want to isolate for batch computing. 
  -----
  Returns: 
    The original dataframe with an added column containing the mapped re288 value for each pitch. 
  """
  df = df.sort_values(by=['pitcher', 'game_date', 'game_pk', 'inning', 'outs_when_up', 'pitch_number'])
  df = df.reset_index(drop=True)

  # CHANGE IN RE288 - CONTINUOUS
  temp_list = []  # Empty list to store continuous change values
  for index in range(len(df)):
    # First pitch in the dataframe
    if index==0: 
      temp_list.append(0)

    # Last pitch in the dataframe
    elif index==len(df)-1:                    
      temp_list.append(df['inning'][index] - 1.96)
    
    # Changes between pitchers
    elif df['pitcher'][index-1]!=df['pitcher'][index]:
      temp_list.append(0)

    # Changes between games
    elif df['game_pk'][index-1]!=df['game_pk'][index]:
      temp_list.append(0)
    
    # Within game changes
    elif df['game_pk'][index-1]==df['game_pk'][index]:
      # Last pitch of a game
      if df['game_pk'][index+1]!=df['game_pk'][index]:
          temp_list.append(df['inning'][index] - 1.96)
      # First pitch of new inning within ongoing game. 
      elif df['inning'][index-1]!=df['inning'][index]: 
        temp_list.append(df['inning'][index] - 1.96)
      # All pitches within the same inning. 
      elif df['inning'][index-1]==df['inning'][index]:  
        temp_list.append(df['re288'][index-1]-df['re288'][index])
    
    # Providing notebook user with updating progress after every 100th loop. 
    if index%100==0:
      clear_output()
      print(f'Season: {year}\nRE288 Continuous Change: {index} of {len(df)} completed ({int((index/len(df))*100)}%)')

  # Add data to the original passed dataframe
  df['re_288_change_raw'] = temp_list
  """--------------------------------------------------------------------------------------------"""
  # CHANGE IN RE288 - ORDINAL
  # Empty list to store ordinal change values. 
  temp_list = []                               

  for index in range(len(df)):  
    # First pitch in the dataframe
    if index==0: 
      temp_list.append(0)
    
    # Last pitch thrown in the dataframe 
    elif index==len(df)-1: 
      if (df['inning'][index] - 1.96) > 0:
        temp_list.append(1)
      elif (df['inning'][index] - 1.96) == 0:
        temp_list.apend(0)
      else:
        temp_list.append(-1)
      
    # Changes between pitchers
    elif df['pitcher'][index-1]!=df['pitcher'][index]:
      temp_list.append(0)

    # Changes between games
    elif df['game_pk'][index-1]!=df['game_pk'][index]:
      temp_list.append(0)

    # Within game changes
    elif df['game_pk'][index-1]==df['game_pk'][index]:
      # Last pitch of a game
      if df['game_pk'][index+1]!=df['game_pk'][index]: 
        if (df['inning'][index] - 1.96) > 0:
          temp_list.append(1)
        elif (df['inning'][index] - 1.96) == 0:
          temp_list.apend(0)
        else:
          temp_list.append(-1)
      # First pitch of a new inning within ongoing game
      elif df['inning'][index-1]!=df['inning'][index]: 
        if (df['inning'][index] - 1.96) > 0:
          temp_list.append(1)
        elif (df['inning'][index] - 1.96) == 0:
          temp_list.apend(0)
        else:
          temp_list.append(-1)
      # All pitches within the same inning
      elif df['inning'][index-1]==df['inning'][index]:
        if df['re288'][index-1]>df['re288'][index]:
          temp_list.append(1)
        elif df['re288'][index-1]<df['re288'][index]:
          temp_list.append(-1)
        else:
          temp_list.append(0)

    # Providing notebook user with updating progress after every 100th loop. 
    if index%100==0:
      clear_output()
      print(f'Season: {year}\nRE288 Ordinal Change: {index} of {len(df)} completed ({int((index/len(df))*100)}%)')
    
  # Add data to the original passed dataframe
  df['re_288_change_ord'] = temp_list

  return df

# Add 288 Game State and RE288 to each year df

In [6]:
# Add re288 to all year dfs
for year in [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
  temp_df = pd.read_csv(f'../Data/02_intermediate/df_{year}.csv')
  drop_list = [i for i in list(temp_df) if 'Unnamed' in i]
  temp_df = temp_df.drop(drop_list, axis=1)
  for col in ['on_1b', 'on_2b', 'on_3b']:
    temp_df[col] = temp_df[col].fillna(0)
  temp_df = add_gs288(temp_df, year)
  temp_df.to_csv(f'../Data/02_intermediate/df_{year}.csv')

# Take a look at the result
temp_df.head(10)

Season: 2019
732400 of 732473 completed (99%)


,game_year,game_pk,pitch_type,game_date,type,balls,strikes,inning_topbot,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,events,description,inning,pitcher,player_name,batter,on_1b,on_2b,on_3b,outs_when_up,re_24,re_24_change_raw,re_24_change_ord,so_change,gs_288,re288
0,2019,566106,FF,2019-05-11,B,0,0,Bot,1,4-Seam Fastball,2,1,2,1,NaN,ball,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,0,1,1.959975
1,2019,566106,FF,2019-05-11,S,1,0,Bot,2,4-Seam Fastball,2,1,2,1,NaN,called_strike,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1,2,1.940063
2,2019,566106,SL,2019-05-11,S,1,1,Bot,3,Slider,2,1,2,1,NaN,foul,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1,6,1.960220
3,2019,566106,SL,2019-05-11,S,1,2,Bot,4,Slider,2,1,2,1,NaN,foul,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1,10,1.996588
4,2019,566106,SL,2019-05-11,S,1,2,Bot,5,Slider,2,1,2,1,NaN,foul,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1,10,1.996588
5,2019,566106,SL,2019-05-11,X,1,2,Bot,6,Slider,2,1,2,1,field_out,hit_into_play,8,595918,A.J. Cole,543760,0.0,0.0,0.0,0,0.461,0.000,0,1,10,1.996588
6,2019,566106,SL,2019-05-11,S,0,0,Bot,1,Slider,2,1,2,1,NaN,swinging_strike,8,595918,A.J. Cole,595777,0.0,0.0,0.0,1,0.243,0.218,1,1,13,1.963263
7,2019,566106,CH,2019-05-11,B,0,1,Bot,2,Changeup,2,1,2,1,NaN,ball,8,595918,A.J. Cole,595777,0.0,0.0,0.0,1,0.243,0.000,0,0,17,1.949780
8,2019,566106,CH,2019-05-11,B,1,1,Bot,3,Changeup,2,1,2,1,NaN,ball,8,595918,A.J. Cole,595777,0.0,0.0,0.0,1,0.243,0.000,0,0,18,1.941381
9,2019,566106,SL,2019-05-11,S,2,1,Bot,4,Slider,2,1,2,1,NaN,swinging_strike,8,595918,A.J. Cole,595777,0.0,0.0,0.0,1,0.243,0.000,0,1,19,1.927357


# Add changes in RE88

In [7]:
# Add RE88 changes to all dfs
for year in [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
  temp_df = pd.read_csv(f'../Data/02_intermediate/df_{year}.csv')
  drop_list = [i for i in list(temp_df) if 'Unnamed' in i]
  temp_df = temp_df.drop(drop_list, axis=1)
  temp_df = temp_df.reset_index(drop=True)
  for col in ['on_1b', 'on_2b', 'on_3b']:
    temp_df[col] = temp_df[col].fillna(0)
  temp_df = re288_change(temp_df, year)
  temp_df.to_csv(f'../Data/02_intermediate/df_{year}.csv')

# Take a look at the result
temp_df.head(20)

Season: 2019
RE288 Ordinal Change: 732400 of 732473 completed (99%)


,game_year,game_pk,pitch_type,game_date,type,balls,strikes,inning_topbot,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,events,description,inning,pitcher,player_name,batter,on_1b,on_2b,on_3b,outs_when_up,re_24,re_24_change_raw,re_24_change_ord,so_change,gs_288,re288,re_288_change_raw,re_288_change_ord
0,2019,567463,FC,2019-04-13,B,0,0,Top,1,Cutter,0,0,0,0,NaN,ball,1,282332,CC Sabathia,544725,0.0,0.0,0.0,0,0.461,0.000,0,0,1,1.959975,0.000000,0
1,2019,567463,FC,2019-04-13,S,1,0,Top,2,Cutter,0,0,0,0,NaN,called_strike,1,282332,CC Sabathia,544725,0.0,0.0,0.0,0,0.461,0.000,0,1,2,1.940063,0.019911,1
2,2019,567463,FC,2019-04-13,X,1,1,Top,3,Cutter,0,0,0,0,field_out,hit_into_play,1,282332,CC Sabathia,544725,0.0,0.0,0.0,0,0.461,0.000,0,1,6,1.960220,-0.020157,-1
3,2019,567463,FC,2019-04-13,S,0,0,Top,1,Cutter,0,0,0,0,NaN,foul,1,282332,CC Sabathia,641313,0.0,0.0,0.0,1,0.243,0.218,1,1,13,1.963263,-0.003043,-1
4,2019,567463,SL,2019-04-13,S,0,1,Top,2,Slider,0,0,0,0,NaN,swinging_strike,1,282332,CC Sabathia,641313,0.0,0.0,0.0,1,0.243,0.000,0,1,17,1.949780,0.013483,1
5,2019,567463,FC,2019-04-13,B,0,2,Top,3,Cutter,0,0,0,0,NaN,ball,1,282332,CC Sabathia,641313,0.0,0.0,0.0,1,0.243,0.000,0,0,21,1.969295,-0.019516,-1
6,2019,567463,SL,2019-04-13,S,1,2,Top,4,Slider,0,0,0,0,strikeout,swinging_strike,1,282332,CC Sabathia,641313,0.0,0.0,0.0,1,0.243,0.000,0,1,22,1.972068,-0.002772,-1
7,2019,567463,FC,2019-04-13,B,0,0,Top,1,Cutter,0,0,0,0,NaN,ball,1,282332,CC Sabathia,547989,0.0,0.0,0.0,2,0.095,0.148,1,0,25,1.976618,-0.004550,-1
8,2019,567463,FC,2019-04-13,B,1,0,Top,2,Cutter,0,0,0,0,NaN,ball,1,282332,CC Sabathia,547989,0.0,0.0,0.0,2,0.095,0.000,0,0,26,1.975826,0.000792,1
9,2019,567463,SL,2019-04-13,S,2,0,Top,3,Slider,0,0,0,0,NaN,called_strike,1,282332,CC Sabathia,547989,0.0,0.0,0.0,2,0.095,0.000,0,1,27,1.930253,0.045573,1
